<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.  
This script retrieves the resources created during the deployment, deletes each one, and purges them if necessary. Finally, it deletes the resource group.

In [ ]:
import os, time, json, utils
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}"

query = "\"{type:type, name:name, location:location}\""
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")
if output.success and output.json_data and output.json_data.get("properties") and output.json_data.get("properties").get("provisioningState") == "Succeeded" and output.json_data.get("properties").get("outputResources"):
    for resource in output.json_data.get("properties").get("outputResources"):
        resource_id = resource.get("id")
        try:
            output = utils.run(f"az resource show --id {resource_id} --query {query} -o json")
            if output.success and output.json_data:
                resource_type = output.json_data.get("type")
                resource_name = output.json_data.get("name")
                resource_location = output.json_data.get("location")
                if resource_type == "Microsoft.ContainerService/ManagedClusters":
                    output = utils.run(f"az aks delete --name {resource_name} --resource-group {resource_group_name} --yes", f"Deleted resource {resource_name}", f"Failed to delete resource {resource_name}")
                elif resource_type == "Microsoft.KeyVault/vaults":
                    output = utils.run(f"az keyvault delete --name {resource_name} --resource-group {resource_group_name}", f"Deleted resource {resource_name}", f"Failed to delete resource {resource_name}")
                    output = utils.run(f"az keyvault purge --name {resource_name} --resource-group {resource_group_name} --location {resource_location}", f"Purged resource {resource_name}", f"Failed to purge resource {resource_name}")
                elif resource_type == "Microsoft.CognitiveServices/accounts":
                    output = utils.run(f"az cognitiveservices account delete --name {resource_name} --resource-group {resource_group_name}", f"Deleted resource {resource_name}", f"Failed to delete resource {resource_name}")
                    output = utils.run(f"az cognitiveservices account purge --name {resource_name} --resource-group {resource_group_name} --location {resource_location}", f"Purged resource {resource_name}", f"Failed to purge resource {resource_name}")
                elif resource_type == "Microsoft.ApiManagement/service":
                    output = utils.run(f"az apim delete --name {resource_name} --resource-group {resource_group_name} --yes", f"Deleted resource {resource_name}", f"Failed to delete resource {resource_name}")
                    output = utils.run(f"az apim deletedservice purge --name {resource_name} --resource-group {resource_group_name} --location {resource_location}", f"Purged resource {resource_name}", f"Failed to purge resource {resource_name}")
                elif resource_type == "Microsoft.Network/virtualNetworks":
                    utils.print_warning(f"Skipping deletion of resource {resource_name} with type {resource_type}")
                else:
                    output = utils.run(f"az resource delete --id {resource_id}", f"Deleted resource {resource_name} with tyoe {resource_type}", f"Failed to delete resource {resource_name} with type {resource_type}")
        except:
            utils.print_warning(f"Failed to delete resource {resource_name} with type {resource_type}")
else:
    utils.print_warning(f"Invalid deployment: {deployment_name}")
output = utils.run(f"az group delete --name {resource_group_name} --yes", f"Deleted resource group {resource_group_name}", f"Failed to delete resource group {resource_group_name}")

